In [1]:
%matplotlib inline
import pandas as pd
import statsmodels.api as sm 
import numpy as np
import pandas.io.data as web
import datetime
import cvxpy as cvx

In [2]:
hedge = "SPXU" # The ticker to be used for hedging. It should have negative beta
FUM = 216309.85 
tickers = ["HMSY", "SONC"] # Tickers to invest in
last_prices = { "HMSY": 8.43, "SONC": 29.26, hedge: 33.55 } # The prices at which the positions will be bought

start = datetime.datetime.now() - datetime.timedelta(days=366)
end = datetime.datetime.now() - datetime.timedelta(days=1)
market = web.DataReader(["SPY"], data_source="yahoo", start=start, end=end)["Adj Close"]
prices = web.DataReader(tickers + [hedge], data_source="yahoo", start=start, end=end)["Adj Close"]
returns = np.log(prices / prices.shift(1))
returns = returns[1:]
market_returns = np.log(market / market.shift(1))
market_returns = market_returns[1:]

In [34]:
# Calculate the beta of each ticker and the hedge
betas = {}
for ticker in returns.columns:
    X = sm.add_constant(market_returns['SPY'].values)
    ols_result = sm.OLS(returns[ticker].values, X).fit()
    betas[ticker] = ols_result.params[1]
    print "{} has R^2: {}".format(ticker, ols_result.rsquared)
betas = pd.Series(betas, index=prices.columns)
print betas

HMSY has R^2: 0.0791844805495
SONC has R^2: 0.155695556327
SPXU has R^2: 0.997658777652
HMSY    1.195514
SONC    1.086335
SPXU   -2.987957
dtype: float64


In [35]:
average_beta = np.average(betas[betas.index - [hedge]])
stock_coef = - betas[hedge] / average_beta
funds_per_stock = FUM * stock_coef / (stock_coef + 1) / (len(betas) - 1)
hedge_funds = FUM / (stock_coef + 1)

positions = betas.copy()
positions[hedge] = np.floor(hedge_funds / last_prices[hedge])

for ticker in (betas.index - [hedge]):
    positions[ticker] = np.floor(funds_per_stock / last_prices[ticker])

positions

HMSY    9284
SONC    2674
SPXU    1781
dtype: float64

In [36]:
abs_positions = np.multiply(prices.ix[-1], positions.T)
print "Absolute positions (in $):"
print abs_positions
print "Relative positions:"
print abs_positions / FUM

Absolute positions (in $):
HMSY    76500.16
SONC    78241.24
SPXU    59948.46
Name: 2015-08-06 00:00:00, dtype: float64
Relative positions:
HMSY    0.353660
SONC    0.361709
SPXU    0.277142
Name: 2015-08-06 00:00:00, dtype: float64


In [15]:
print "Total beta: {}".format(np.sum(np.multiply(betas, abs_positions.T / FUM)))

Total beta: 0.000351186027443


In [20]:
print returns.cov()
print returns.mean()
print returns.std()

          HMSY      SONC      SPXU
HMSY  0.001077  0.000109 -0.000187
SONC  0.000109  0.000490 -0.000220
SPXU -0.000187 -0.000220  0.000823

[3 rows x 3 columns]
HMSY   -0.002330
SONC    0.000432
SPXU   -0.001449
dtype: float64
HMSY    0.032819
SONC    0.022132
SPXU    0.028682
dtype: float64


In [19]:
974.14745711  * 0.00107708 

1.0492347431040387